In [1]:
import numpy as np
import pandas as pd
import os
import folium
from folium import plugins
import pygeohash as geo
from datetime import datetime
import calendar
from datetime import timezone
import datetime
import pytz
import geopandas as gpd

In [2]:
def process_maps(file):
    types = {'venue_name':'str','venue_category':'str','name':'str','poi_place_id':'str','speed':'float', 'types':'str', 'vicinity':'str','publisher_id':'str'}
    f = '/'.join(['Final_integrated_files_fixed/DC_data',file])
    data_orig= pd.read_csv(f,dtype=types)
    
    data_orig['dwell_estimated'] = data_orig['dwell_end']-data_orig['dwell_start']
    
    data_orig.sort_values('location_at')
    deduped = data_orig.drop_duplicates(subset = ['advertiser_id', 'place_id_latitude', 'place_id_longitude', 'dwell_start', 'dwell_end'], keep = 'first')
    
    geom = gpd.read_file('Census_Data/cbg.geojson')
    
    geom = geom.rename({'CensusBlockGroup':'census_block_group_stop'},axis=1)
    geom = geom[['census_block_group_stop','geometry']]
    
    deduped.loc[deduped.place_id_latitude.isna(),'place_id_latitude'] = deduped.loc[deduped.place_id_latitude.isna(),'latitude']
    deduped.loc[deduped.place_id_longitude.isna(),'place_id_longitude'] = deduped.loc[deduped.place_id_longitude.isna(),'longitude']
    
    deduped = gpd.GeoDataFrame(deduped, geometry = gpd.points_from_xy(deduped.place_id_longitude,deduped.place_id_latitude))
    deduped = gpd.sjoin(deduped,geom,op='within')
    
    #deduped = deduped.rename(columns={deduped.columns[0]: 'drop'})
    #deduped = deduped.drop(columns = ['drop'])
    
    #data_orig['dwell_start_est']= pd.to_datetime(data_orig['dwell_start'],unit='s')
    #data_orig['dwell_start_est'] = data_orig['dwell_start_est'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    
    #data_orig['month'] = pd.DatetimeIndex(data_orig['dwell_start_est']).month
    #data_orig['day'] = pd.DatetimeIndex(data_orig['dwell_start_est']).strftime('%A')
    #data_orig['hour']= pd.DatetimeIndex(data_orig['dwell_start_est']).hour
    
    census = pd.read_csv('user_list_census.csv')
    merged = deduped.merge(census,on='advertiser_id',how='inner')
    
    cwd = os.getcwd()
    path = cwd + "/Adjusted_Location_Data/new_" + file
    merged.to_csv(path)

In [3]:
def spark_process_maps(x):
    return [process_maps(entry) for entry in x]

In [4]:
#files = os.listdir(os.getcwd() + '/DC-i1-revised-g6-poi')
files = os.listdir(os.getcwd() + '/Final_integrated_files_fixed/DC_data')
files = [i for i in files]

In [5]:
rdd =sc.parallelize(files).mapPartitions(spark_process_maps).collect()